# PowerCO Algorithm


**Author:** Ingrid Cadu<br>
**Last update:** Apr, 07, 2022<br>
<br>
This notebook contains the model that will evaluate the churn on PowerCo customers, and as asked It will also check the hyposthesis about the 20% off and its effects on people predicted as possible churn.<br>

**Background**<br>
- The SME team suggestions:
        1. Feature engineering is one of the keys to unlocking predictive insight through mathematical modeling. Based on the data that is available and was cleaned, identify what you think could be drivers of churn for our client and build those features to later use in your model.

        2. First focus on building on top of the feature that your colleague has already investigated: “the difference between off-peak prices in December and January the preceding year”. After this, if you have time, feel free to get creative with making any other features that you feel are worthwhile.

        3. Once you have a set of features, you must train a Random Forest classifier to predict customer churn and evaluate the performance of the model with suitable evaluation metrics. Be rigorous with your approach and give full justification for any decisions made by yourself as the intern data scientist. 

- **Extra Task:**<br>
    Recall that the hypotheses under consideration is that churn is driven by the customers’ price sensitivities and that it would be possible to predict customers likely to churn using a predictive model. If you’re eager to go the extra mile for the client, when you have a trained predictive model, remember to investigate the client’s proposed discounting strategy, with the head of the SME division suggesting that offering customers at high propensity to churn a 20% discount might be effective.

**Prior Objective**<br>
Build your models and test them while keeping in mind you would need data to prove/disprove the hypotheses, as well as to test the effect of a 20% discount on customers at high propensity to churn.

# Loading the libraries and data

In [ ]:
#Libraries
#---------------------

#basic
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

In [ ]:
#Data loading
client_df = pd.read_csv("client_data.csv")
price_df = pd.read_csv("price_data.csv")

# Data Calcs

In [ ]:
#Let's incresing more features

client_df['date_activ'] = pd.to_datetime(client_df['date_activ'], format='%Y-%m-%d')
client_df['date_end'] = pd.to_datetime(client_df['date_end'], format='%Y-%m-%d')
client_df['date_modif_prod'] = pd.to_datetime(client_df['date_modif_prod'], format='%Y-%m-%d')

#Date
client_df['diff_end_activ'] = client_df['date_end'].dt.day - client_df['date_activ'].dt.day

#Let's add a new cons_12m -> cons_fore_12m
client_df['cons_fore_12m'] = client_df['cons_12m'] - client_df['forecast_cons_12m']
client_df['cons_fore_12m'] = client_df['cons_fore_12m']

#Let's add a new col -> the time between renewal and end
client_df['modify_b4_end'] = (client_df['date_end'] - client_df['date_modif_prod'])

#Let's add a new col -> imp + net
client_df['imp_net'] = client_df['imp_cons'] + client_df['net_margin']
client_df['imp_net'] = client_df['imp_net']

In [ ]:
#grouping prices by its mean per year + churn col
price_off_peak = price_df.groupby('id').agg({'price_off_peak_var':'mean',
                                                      'price_off_peak_fix':'mean'})

price_peak = price_df.groupby('id').agg({'price_peak_var':'mean',
                                         'price_peak_fix':'mean'})

POP = pd.merge(price_off_peak, client_df, on=['id'], how='left')
PP = pd.merge(price_peak, client_df, on=['id'], how='left')

POP = POP.loc[:,['id', 'price_off_peak_var', 'price_off_peak_fix', 'churn']].copy()
PP = PP.loc[:,['id', 'price_peak_var', 'price_peak_fix', 'churn']].copy()

In [ ]:
def miss_u(dt):
    miss = dt.isna().sum()
    total = np.product(dt.shape)
    total_miss = miss.count().sum()
    print(f'There are {round((total_miss/total)*100, 2)}% of missing values between price and client id after merging the prices off peak dataset')

In [ ]:
miss_u(POP)

In [ ]:
miss_u(PP)

# Data Analysis

In [ ]:
client_df['churn'].value_counts()
#round(client_df['churn'].value_counts()/len(client_df),2)

### Trends and comparisons

In [ ]:
#Consume in 12 months

fig = px.histogram(client_df, x="cons_12m", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=client_df.columns,
                  height=400, width=500)
fig.show()

In [ ]:
fig = px.histogram(client_df, x="diff_end_activ", color="churn", height=400, width=500)
fig.show()

In [ ]:
fig = px.histogram(client_df, x="cons_fore_12m", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=client_df.columns,
                  height=400, width=500)
fig.show()

In [ ]:
x = client_df[client_df['churn']==1]
x['modify_b4_end'].value_counts()

In [ ]:
x[x['modify_b4_end']=='2557 days']
len(x)

In [ ]:
fig = px.histogram(client_df, x="imp_net", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=client_df.columns,
                  height=400, width=500)
fig.show()

### Price Dataset

In [ ]:
POP.dropna(axis=0, inplace=True)
PP.dropna(axis=0, inplace=True)

In [ ]:
fig = px.histogram(POP, x="price_off_peak_var", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=POP.columns,
                  height=400, width=500)
fig.show()

In [ ]:
#Comparisons

z = POP[POP['churn']==1]
#z.price_off_peak_var.min()
#len(z[z["price_off_peak_var"]>0.1031])
#POP["price_off_peak_var"].mean()#42.92 var=.1423
#z.price_off_peak_var.value_counts()

In [ ]:
z_score_off_var = (0.1418-0.1423)/0.022 #proportion of .49202 ~56.4% (801) below the mean var
z_score_off_fix = (43.16-42.92)/4.55 #proportion of .51994 ~56% (795) above the mean fix

In [ ]:
fig = px.histogram(POP, x="price_off_peak_fix", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=POP.columns,
                  height=400, width=500)
fig.show()

In [ ]:
fig = px.histogram(PP, x="price_peak_fix", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=PP.columns,
                  height=400, width=500)
fig.show()

In [ ]:
fig = px.histogram(PP, x="price_peak_var", color="churn", marginal="rug", # can be `box`, `violin`
                         hover_data=PP.columns,
                  height=400, width=500)
fig.show()

In [ ]:
#Comparisons
q = PP[PP['churn']==1]
#PP.price_peak_fix.std()
len(q[q["price_peak_fix"]>11.19])
#PP["price_off_peak_var"].mean()#42.92 var=.1423

In [ ]:
z_score_var = (0.056-0.052)/0.049 #proportion of .78814 ~55.5% (788) below the mean var
z_score_fix = (11.19-9.46)/12.05 #proportion of .55567 ~45.3% (643) above the mean fix

### Considerations***
1. ~79.42% churn people was consuming between 0 & 50K in 12m, and ~9.72% was between 50K and 100K.
2. ~72.16% churn people renewal their contract in the right day, and just ~3.87% pass over one day to renewal.
3. churn people started with whose had ~4 years of contract; client data has 1061 people with this same time.
4. The highest rate of churn was between the values 0.1235 & 0.1245 (11.06%), the second rate was between 0.1495 & 0.1505 (~9.51%). Price off var.
5. Churn rate was higher ~28% between 40.65 & 40.69 passing over the client with 25%. However, the max churn was between the values of 44.
6. The only flag was who was paying between 0.1025 & 0.1075 churned more. Price peak var

# Hypotesis Test

In [ ]:
#Merging POP and client_df

DP = pd.merge(client_df, POP, on=['id'], how='left')
CP = pd.merge(DP, PP, on=['id'], how='left')
CP.drop(["churn_y"], axis=1, inplace=True)

In [ ]:
#Filter by consume

future_churn = CP[CP['churn_x']==0]
future_churn1 = future_churn[(future_churn['cons_12m']>0) & (future_churn['cons_12m'] <= 50000)] 

In [ ]:
#Filter by days of renewal

future_churn2 = future_churn1[future_churn1['diff_end_activ']==0]

In [ ]:
#Filter by years of contract

future_churn3 = future_churn2[future_churn2['num_years_antig']==4]

In [ ]:
#Filtering by Price - First Rate ~ 135 people

price_off = future_churn3[(future_churn3['price_off_peak_var'] >= 0.1235)&(future_churn3['price_off_peak_var'] <= 0.1245)]

In [ ]:
#Filtering by price - second rate ~367 people

price_off2 = future_churn3[(future_churn3['price_off_peak_var'] >= 0.1495)&(future_churn3['price_off_peak_var'] <= 0.1505)]

In [ ]:
#Filtering by Fix Price - First Rate ~ 181 people

price_off_fix = future_churn3[(future_churn3['price_off_peak_fix'] >= 40.65)&(future_churn3['price_off_peak_fix'] <= 40.69)]

In [ ]:
#Filtering by Fix Price - Second Rate ~ 906 people

price_off_fix2 = future_churn3[(future_churn3['price_off_peak_fix'] >= 44.3)&(future_churn3['price_off_peak_fix'] <= 44.49)]

In [ ]:
#The first group - with lowest values of price off peak and price_peak_var 0.1025 & 0.1075 ~ 26 people

price_test = price_off[(price_off['price_off_peak_fix'] > 40.65)&(price_off['price_off_peak_fix'] <= 40.69)]
x = price_test[(price_test['price_peak_var'] > 0.1025)&(price_test['price_peak_var'] <=0.1075)]
x.shape

In [ ]:
price_test_h = price_off[(price_off['price_off_peak_fix'] > 44.3)&(price_off['price_off_peak_fix'] <= 44.9)]
w = price_test_h[(price_test['price_peak_var'] >= 0.1025)&(price_test_h['price_peak_var'] <=0.1075)]
w.shape

In [ ]:
price_test_hsec = price_off2[(price_off2['price_off_peak_fix'] >= 40.65)&(price_off2['price_off_peak_fix'] <= 40.69)]
price_test_hsec[(price_test_hsec['price_peak_var'] >= 0.1025)&(price_test_hsec['price_peak_var'] <=0.1075)]
price_test_hsec.shape

In [ ]:
#The second group - with highest price off and lowest price peak var ~ 230 people

price_test_sec = price_off2[(price_off2['price_off_peak_fix'] >= 44.3)&(price_off2['price_off_peak_fix'] <= 44.49)]
u = price_test_sec[(price_test_sec['price_peak_var'] >= 0.1025)&(price_test_sec['price_peak_var'] <=0.1075)]
u.shape

In [ ]:
w = price_test[price_test['price_peak_var']<= 0.0025]
w.shape

In [ ]:
#Second Option: New test - Adding PP as a last feature - price_peak_var < 0.0025 ~208 people

y = price_test_sec[price_test_sec['price_peak_var'] <= 0.0025]
y.shape

In [ ]:
#Filtering by Impost and net margin
net_y = y[y['imp_net']<75]
net_y.shape

In [ ]:
#Filtering by Impost and net margin
net_x = x[x['imp_net']<75]
net_x.shape

In [ ]:
final_data = pd.concat([net_y, net_x])
final_data

In [ ]:
#The proportion of future churn date-end month ~again 8th & 9th is in the top (it's a prior)

final_data['date_end'].dt.month.value_counts()[:4]

# Recomendation

The data showed that churn people were paying above the mean per client. Even though the churning reason was pointed to price, the machine considered consumption an important feature and as it was not computed, the main factors to build a pattern is non-understandable, however their little patterns were statistically applied and 103 current clients showed the same behaviour as churn people.<br>
The solution of applying a discount is a good aproach but as we saw early the price is highly above the estimated value so there are some things to consider before applying a discount:
- More Quality of Service
- Build a relationship with customers
- Priorize customers in specific geo areas - segmentation